# count min sketch source :
https://github.com/rafacarrascosa/countminsketch

In [3]:
import pip
pip.main(['install', "countminsketch"])

0

In [4]:
#from count_min_sketch import CountMinSketch
from countminsketch import CountMinSketch
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import math

In [5]:
np.random.seed(10)
p= 10000
w = 4

#column_names = ['A','B','C','D','E','F','G','H','I','J']
#data_frame = pd.read_fwf("F:\\IIT HYDERABAD NOTES\\Topics in DBMS\\yeast.data",
#                         sep = '\t',names = column_names)
#print data_frame.head()

In [72]:
"""data_frame['A'] = data_frame['A'].astype('category')
data_frame['J'] = data_frame['J'].astype('category')
cat_cols = ['A','J']

data_frame[cat_cols] = data_frame[cat_cols].apply(lambda x : x.cat.codes)
print data_frame.head()
print(data_frame.shape)

#data_list = data_frame.values.tolist()
"""

"data_frame['A'] = data_frame['A'].astype('category')\ndata_frame['J'] = data_frame['J'].astype('category')\ncat_cols = ['A','J']\n\ndata_frame[cat_cols] = data_frame[cat_cols].apply(lambda x : x.cat.codes)\nprint data_frame.head()\nprint(data_frame.shape)\n\n#data_list = data_frame.values.tolist()\n"

In [6]:
waveform_data = pd.read_csv("sanitized_waveform.data",header = None)

#count = 0

#for i in range(0,len(waveform_data)):
#    waveform_data[i] = tuple(waveform_data[i])
    

print(waveform_data.head())

     0     1     2     3     4     5     6     7     8     9  ...    12    13  \
0  2.06  2.38 -1.49 -0.50 -1.03  1.72  2.13  1.56  4.33  2.53 ...  4.01  2.22   
1  1.65  1.58  1.83  1.80  3.59  2.79  3.52  4.77  5.15  5.14 ...  0.29  2.28   
2  0.21  0.25  0.27 -0.43 -0.02  0.58  0.22 -0.35  0.06  2.71 ...  3.21  4.92   
3 -0.62  0.87  1.69  0.40  0.92  3.56  4.17  5.26  2.19  5.30 ...  2.45  1.99   
4  0.39  0.61 -0.08 -0.37  1.09 -0.81  0.56 -0.10  3.50  4.20 ...  4.85  4.64   

     14    15    16    17    18    19    20  21  
0  4.22  2.39  2.47  0.80 -0.14 -0.46 -0.33   2  
1  2.14 -0.24 -0.72  0.13 -0.39 -0.06 -1.18   1  
2  4.23  5.47  3.97  3.60  1.86  0.40  0.26   2  
3  2.00  0.12 -0.53 -0.79 -1.31  0.11 -1.09   1  
4  4.25  3.49  1.99  3.30  1.61  0.56  1.80   2  

[5 rows x 22 columns]


In [7]:
cm_sketch = CountMinSketch(p,w)

In [75]:
#print(data_frame.ix[1].values.tolist())
#a[1] = data_frame.ix[1].values.tolist()
#a = CountMinSketch(p,w)
#a[1] = 2
#a[1] += 1
#for i in a.get_columns(1):
#    print i
#
#print(a.query(1))

#a.add(tuple([1,2,3]))
#print a[tuple([1,2,3])]
#print a[tuple([1,2,3]]
#print(a.tables)
#p = [4,5,6]
#p = np.vectorize(p)
#print(p[0])




In [8]:
seed_np = np.random.seed(1)
s = 1000.0                                                      #sample size
sqrt_s = np.sqrt(s)

f = np.random.uniform(1.0/sqrt_s,1.0 - (1.0/sqrt_s))            #locality parameter

print(f)

0.422269993919


In [9]:
d = waveform_data.shape[1]                                         #dimension of the dataset

random_vector = np.random.uniform(0,f,d)
print(random_vector)

[  3.04171419e-01   4.82970534e-05   1.27665974e-01   6.19706091e-02
   3.89918179e-02   7.86520983e-02   1.45919926e-01   1.67542999e-01
   2.27526139e-01   1.77013265e-01   2.89347634e-01   8.63340503e-02
   3.70802645e-01   1.15649588e-02   2.83118311e-01   1.76215296e-01
   2.35917950e-01   5.92811917e-02   8.36523146e-02   3.38130404e-01
   4.08867810e-01   1.32349626e-01]


In [10]:
np.random.seed(1)
base =  np.maximum(2, 1/f)
r = int(np.random.uniform(1.0+(0.5*math.log(s,base)),math.log(s,base)))
waveform_randomly = waveform_data.sample(n=r,axis = 1)#waveform_data.sample(n=r,axis = 1)
print(waveform_randomly.head())

     19  21    10    1     2     3 
0 -0.46   2  5.44  2.38 -1.49 -0.50
1 -0.06   1  3.19  1.58  1.83  1.80
2  0.40   2 -0.99  0.25  0.27 -0.43
3  0.11   1  4.15  0.87  1.69  0.40
4  0.56   2  2.40  0.61 -0.08 -0.37


In [11]:
print(waveform_randomly.shape)

(3509, 6)


In [12]:
sample_data = waveform_randomly.sample(n = int(s),random_state=12)
print(sample_data.shape)
base = np.maximum(2,1/f)
print(base)


(1000, 6)
2.36815311152


In [81]:
#sample 1000 data from randomly_waveform
#waveform_randomly_train = waveform_randomly.sample(n=1000,axis = 0)
#print(waveform_randomly_train.head())


#for i in range(0,len(waveform_data)):
#    waveform_randomly[i] = tuple(waveform_data[i])

In [13]:
col_max = np.max(sample_data,axis = 0)
col_min = np.min(sample_data,axis = 0)
print(col_max)
print(col_min)

19    3.59
21    2.00
10    8.05
1     3.40
2     4.72
3     5.75
dtype: float64
19   -3.06
21    0.00
10   -0.17
1    -3.25
2    -3.56
3    -3.05
dtype: float64


In [14]:
normalized_waveform_sample = (sample_data - col_min)/(col_max-col_min)
print(normalized_waveform_sample.head())

            19   21        10        1         2         3 
3288  0.332331  1.0  0.350365  0.464662  0.342995  0.355682
3366  0.569925  0.5  0.473236  0.676692  0.682367  0.428409
2037  0.497744  0.5  0.354015  0.412030  0.485507  0.646591
2881  0.824060  0.5  0.330900  0.612030  0.728261  0.600000
2999  0.569925  1.0  0.517032  0.584962  0.359903  0.351136


In [15]:
indexes_sample = sample_data.index
waveform_sample_rows = waveform_data.ix[indexes_sample]
print(waveform_sample_rows.head())

        0     1     2     3     4     5     6     7     8     9  ...    12  \
3288 -0.06 -0.16 -0.72  0.08 -0.04  2.06  0.87 -1.01  0.76  1.76 ...  4.91   
3366  0.18  1.25  2.09  0.72  1.22  2.69  3.77  3.21  6.49  3.71 ...  3.62   
2037  0.53 -0.51  0.46  2.64  1.88  2.81  1.55  2.43  3.74  4.34 ...  3.27   
2881  0.29  0.82  2.47  2.23  4.88  5.53  6.19  4.67  4.83  4.34 ...  3.06   
2999 -1.24  0.64 -0.58  0.04 -0.40 -0.11  2.24  1.73  0.16  1.46 ...  4.30   

        13    14    15    16    17    18    19    20  21  
3288  4.74  4.93  3.81  4.34  3.22  2.22 -0.85  0.55   2  
3366  0.35  1.69  1.23  0.22  0.16 -1.07  0.73 -1.02   1  
2037  1.37 -1.12  1.97 -0.04 -0.85 -0.14  0.25  1.31   1  
2881  0.99 -0.71  1.08 -0.79 -0.44 -0.82  2.42  0.71   1  
2999  3.10  3.77  3.68  2.44  2.35  1.00  0.73 -0.54   2  

[5 rows x 22 columns]


In [26]:
included_cols = waveform_randomly.columns
Y_cap = waveform_sample_rows

print(included_cols)

for col in Y_cap:
    if col not in included_cols:
        Y_cap[col] = -1
    else:
        Y_cap[col] = np.floor((Y_cap[col] + random_vector[col] )/ f)
        
print(Y_cap.head())

Int64Index([19, 21, 10, 1, 2, 3], dtype='int64')
      0   1   2   3   4   5   6   7   8   9  ...  12  13  14  15  16  17  18  \
3288  -1  -3  -7   2  -1  -1  -1  -1  -1  -1 ...  -1  -1  -1  -1  -1  -1  -1   
3366  -1  16  30  10  -1  -1  -1  -1  -1  -1 ...  -1  -1  -1  -1  -1  -1  -1   
2037  -1  -7   8  36  -1  -1  -1  -1  -1  -1 ...  -1  -1  -1  -1  -1  -1  -1   
2881  -1  10  35  30  -1  -1  -1  -1  -1  -1 ...  -1  -1  -1  -1  -1  -1  -1   
2999  -1   8  -5   1  -1  -1  -1  -1  -1  -1 ...  -1  -1  -1  -1  -1  -1  -1   

      19  20  21  
3288  -5  -1  29  
3366  16  -1  16  
2037  10  -1  16  
2881  39  -1  16  
2999  16  -1  29  

[5 rows x 22 columns]


In [21]:
#normlized_waveform_sorted  = normalized_waveform.sort_index(axis = 1)
#print(normlized_waveform_sorted.head())

In [27]:
#frame_to_hash = (Y_cap+random_vector)/f
#print(frame_to_hash.head())

In [28]:
data_tuples = [tuple(x) for x in Y_cap.to_records(index=False)]

print(data_tuples[0])

(-1, -3.0, -7.0, 2.0, -1, -1, -1, -1, -1, -1, 42.0, -1, -1, -1, -1, -1, -1, -1, -1, -5.0, -1, 29.0)


In [29]:
for i in data_tuples:
    cm_sketch.add(i)

In [30]:
#included_cols_for_whole_data = waveform_randomly.columns
Y_whole = waveform_data
print(included_cols)

for col in Y_cap:
    if col not in included_cols:
        Y_whole[col] = -1
    else:
        Y_whole[col] = np.floor(Y_whole[col] + random_vector[col] / f)
        
print(Y_whole.head())


Int64Index([19, 21, 10, 1, 2, 3], dtype='int64')
   0   1   2   3   4   5   6   7   8   9  ...  12  13  14  15  16  17  18  19  \
0  -1   2  -2  -1  -1  -1  -1  -1  -1  -1 ...  -1  -1  -1  -1  -1  -1  -1   0   
1  -1   1   2   1  -1  -1  -1  -1  -1  -1 ...  -1  -1  -1  -1  -1  -1  -1   0   
2  -1   0   0  -1  -1  -1  -1  -1  -1  -1 ...  -1  -1  -1  -1  -1  -1  -1   1   
3  -1   0   1   0  -1  -1  -1  -1  -1  -1 ...  -1  -1  -1  -1  -1  -1  -1   0   
4  -1   0   0  -1  -1  -1  -1  -1  -1  -1 ...  -1  -1  -1  -1  -1  -1  -1   1   

   20  21  
0  -1   2  
1  -1   1  
2  -1   2  
3  -1   1  
4  -1   2  

[5 rows x 22 columns]


In [39]:
print(Y_whole.shape)

(3509, 22)


In [43]:
for i in Y_whole:
    cm_sketch.add(i)